In [1]:
!nvidia-smi

Sun Feb  6 05:36:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install --ignore-installed PyYAML
import yaml
print(yaml.__version__)

     |████████████████████████████████| 596 kB 4.2 MB/s 
6.0


In [3]:
import os
import shutil
import json

from tqdm import tqdm
from glob import glob

from PIL import Image

import torch
import random
import numpy as np

In [4]:
# Google Drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!git clone https://github.com/ultralytics/yolov5 
!pip -q install -r /content/yolov5/requirements.txt
!pip -q install wandb

Cloning into 'yolov5'...
remote: Enumerating objects: 10878, done.
remote: Total 10878 (delta 0), reused 0 (delta 0), pack-reused 10878
Receiving objects: 100% (10878/10878), 10.95 MiB | 12.54 MiB/s, done.
Resolving deltas: 100% (7519/7519), done.
     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 143 kB 71.0 MB/s 
     |████████████████████████████████| 180 kB 71.5 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 


### Dataset 불러오기

In [6]:
!tar -xvf /content/drive/MyDrive/AI_Factory_YOLO/Train_Dataset_mini.tar 
!tar -xvf /content/drive/MyDrive/AI_Factory_YOLO/Valid_Dataset_mini.tar 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
content/mini_data/Validation/cow_image/livestock_cow_bbox_001864.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_001540.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_010826.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_007607.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_009472.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_008591.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_010281.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_012614.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_011942.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_011562.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_000206.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_009305.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_008626.jpg
content/mini_data/Validation/cow_image/livestock_cow_bbox_009936.jp

In [7]:
# Train Image
if not os.path.exists('/content/images'):
    os.mkdir('/content/images')
if not os.path.exists('/content/images/train'):
    os.mkdir('/content/images/train')

source_cow = '/content/content/mini_data/Training/cow_image/'
source_pig = '/content/content/mini_data/Training/pig_image/'
destination = '/content/images/train'

cow = os.listdir(source_cow)
pig = os.listdir(source_pig)

for c in tqdm(cow):
    shutil.move(source_cow + c, destination)
for p in tqdm(pig):
    shutil.move(source_pig + p, destination)

100%|██████████| 1721/1721 [00:00<00:00, 26919.95it/s]


In [8]:
# Valid Image
if not os.path.exists('/content/images'):
    os.mkdir('/content/images')
if not os.path.exists('/content/images/valid'):
    os.mkdir('/content/images/valid')

source_cow = '/content/content/mini_data/Validation/cow_image/'
source_pig = '/content/content/mini_data/Validation/pig_image/'
destination = '/content/images/valid'

cow = os.listdir(source_cow)
pig = os.listdir(source_pig)

for c in tqdm(cow):
    shutil.move(source_cow + c, destination)
for p in tqdm(pig):
    shutil.move(source_pig + p, destination)

100%|██████████| 861/861 [00:00<00:00, 28779.62it/s]


In [9]:
# Train label
if not os.path.exists('/content/labels_json'):
    os.mkdir('/content/labels_json')
if not os.path.exists('/content/labels_json/train'):
    os.mkdir('/content/labels_json/train')

source_cow = '/content/content/mini_data/Training/cow_label/'
source_pig = '/content/content/mini_data/Training/pig_label/'
destination = '/content/labels_json/train'

cow = os.listdir(source_cow)
pig = os.listdir(source_pig)

for c in tqdm(cow):
    shutil.move(source_cow + c, destination)
for p in tqdm(pig):
    shutil.move(source_pig + p, destination)

100%|██████████| 1721/1721 [00:00<00:00, 17277.49it/s]


In [10]:
# Valid label
if not os.path.exists('/content/labels_json'):
    os.mkdir('/content/labels_json')
if not os.path.exists('/content/labels_json/valid'):
    os.mkdir('/content/labels_json/valid')

source_cow = '/content/content/mini_data/Validation/cow_label/'
source_pig = '/content/content/mini_data/Validation/pig_label/'
destination = '/content/labels_json/valid'

cow = os.listdir(source_cow)
pig = os.listdir(source_pig)

for c in tqdm(cow):
    shutil.move(source_cow + c, destination)
for p in tqdm(pig):
    shutil.move(source_pig + p, destination)

100%|██████████| 861/861 [00:00<00:00, 25785.58it/s]


In [11]:
train_image = glob('/content/images/train/*.jpg')
train_label = glob('/content/labels_json/train/*.json')

valid_image = glob('/content/images/valid/*.jpg')
valid_label = glob('/content/labels_json/valid/*.json')

test_image = glob('/content/images/test/*.jpg')
test_label = glob('/content/labels_json/test/*.json')

In [12]:
print('Train Data의 개수:', len(train_image))
print('Train Data의 개수:', len(train_label))

print('Valid Data의 개수:', len(valid_image))
print('Valid Data의 개수:', len(valid_label))

print('Test Data의 개수', len(test_image))
print('Test Data의 개수', len(test_label))

Train Data의 개수: 6582
Train Data의 개수: 6582
Valid Data의 개수: 3291
Valid Data의 개수: 3291
Test Data의 개수 0
Test Data의 개수 0


### YOLOv5 Setting에 맞도록 Label 변경

In [13]:
# x1, y1, x2, y2 -> x, y , w, h로 변경
# 이미지 크기에 맞게 정규화
if not os.path.exists('/content/labels'):
    os.mkdir('/content/labels')
if not os.path.exists('/content/labels/test'):
    os.mkdir('/content/labels/test')
if not os.path.exists('/content/labels/train'):
    os.mkdir('/content/labels/train')
if not os.path.exists('/content/labels/valid'):
    os.mkdir('/content/labels/valid')

train_label = glob('/content/labels_json/train/*.json')
valid_label = glob('/content/labels_json/valid/*.json')
train_label.sort()
valid_label.sort()

In [14]:
def convert_coordinate(width, height, x1, y1, x2, y2):
    dw = 1./ width
    dh = 1./ height
    x = (x1 + x2) / 2.0
    y = (y1 + y2) / 2.0
    w = x2 - x1
    h = y2 - y1
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x, y, w, h)

In [15]:
# Train Label 변경
for label in tqdm(train_label):
    with open(label, 'r') as f:
        json_data = json.load(f)
    
    height = json_data['label_info']['image']['height']
    width = json_data['label_info']['image']['width']
    box_info = json_data['label_info']['annotations']
    
    image_name = label.replace('labels_json', 'labels')
    image_name = image_name.replace('json', 'txt')
    
    with open(image_name, 'w') as f:
        for i in range(len(box_info)):
            x1, y1, x2, y2 = box_info[i]['bbox']
            if 'cow' in image_name:
                category = 1
            else:
                category = 0
            x, y, w, h = convert_coordinate(width, height, x1, y1, x2, y2)
            data = str(category) + ' ' + str(x) + ' ' + str(y) +  ' ' + str(w) + ' ' + str(h) + '\n'
            f.write(data)

100%|██████████| 6582/6582 [00:04<00:00, 1644.20it/s]


In [16]:
# Valid Label 변경
for label in tqdm(valid_label):
    with open(label, 'r') as f:
        json_data = json.load(f)
    
    height = json_data['label_info']['image']['height']
    width = json_data['label_info']['image']['width']
    box_info = json_data['label_info']['annotations']
    
    image_name = label.replace('labels_json', 'labels')
    image_name = image_name.replace('json', 'txt')
    
    with open(image_name, 'w') as f:
        for i in range(len(box_info)):
            x1, y1, x2, y2 = box_info[i]['bbox']
            if 'cow' in image_name:
                category = 1
            else:
                category = 0
            x, y, w, h = convert_coordinate(width, height, x1, y1, x2, y2)
            data = str(category) + ' ' + str(x) + ' ' + str(y) +  ' ' + str(w) + ' ' + str(h) + '\n'
            f.write(data)

100%|██████████| 3291/3291 [00:00<00:00, 3986.19it/s]


In [17]:
# 리스트 이름을 txt파일로 생성
train_list = glob('/content/images/train/*.jpg')
valid_list = glob('/content/images/valid/*.jpg')
test_list = glob('/content/images/test/*.jpg')
 
with open('/content/train.txt', 'w') as f:
	f.write('\n'.join(train_list) + '\n')
    
with open('/content/valid.txt', 'w') as f:
	f.write('\n'.join(valid_list) + '\n')
 
with open('/content/test.txt', 'w') as f:
  f.write('\n'.join(test_list) + '\n')

In [18]:
 # coco.yaml을 바탕으로 우리 데이터셋 이름대로 변경하여 data.yaml로 저장
with open('/content/yolov5/data/coco.yaml', 'r') as f:
	data = yaml.load(f, Loader=yaml.FullLoader)
     
data['path'] = '/content' 
data['train'] = 'train.txt'
data['val'] = 'valid.txt'
data['test'] = 'test.txt'
data['nc'] = 2
data['names'] = ['cow','pig']
 
with open('/content/data.yaml', 'w') as f:
	yaml.dump(data, f)

### HyperParameter Tuning

In [19]:
source = '/content/drive/MyDrive/AI_Factory/exp/hyp_train.py'
destination = '/content/yolov5'
shutil.copy(source, destination)

'/content/yolov5/hyp_train.py'

In [ ]:
!python /content/yolov5/hyp_train.py --batch 10 --epochs 7 --data '/content/data.yaml' --weights yolov5m6.pt --img 1280 --evolve 7

In [ ]:
source = '/content/yolov5/runs/evolve/exp/evolve.csv'
destination = '/content/drive/MyDrive/AI_Factory/exp/evolve_hsv.csv'
shutil.move(source,destination)

source = '/content/yolov5/runs/evolve/exp/hyp_evolve.yaml'
destination = '/content/drive/MyDrive/AI_Factory/exp/hyp_evolve_hsv.yaml'
shutil.move(source,destination)